##  Deprecated
must be updated to use the new package architecture

In [53]:
#external libraries
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point,Polygon
import matplotlib.pyplot as plt
import folium
from folium.plugins import TimestampedGeoJson
from datetime import datetime
from src import API,Preprocessor
from src.preprocessing import remove_duplicates_Cleaner,moving_average_Filter,moving_max_Filter,moving_max_averaged_Filter,blank_Filter


import matplotlib as mpl
import matplotlib.colors as mcolors

%matplotlib inline
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
#local imports
from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader
from src.visualization import MapCreation
from func.utils.getMarkerDictList import getMarkerDictList
from func.FoliumFeatureCreation.RawDataFeatures import RawDataFeatures


In [55]:
MapDataPath="data/cartePalaiseau.csv"
ModuleMapPath="data/carteModulePalaiseau.csv"

saving_output=False

sampling_time=2000#ms #si supérieur à 2000ms il faut implémenter la moyenne dans la liste des rssi dans marker dict list

### Data loading


In [56]:
#Map Data
map_gdf=gdfLoader(MapDataPath)
#marker data
marker_gdf=ReceiverLoader(ModuleMapPath)
#rssi data
import ast
#API call
callApi=API()
# start="2023-07-12 11:03:00.000000"
# end="2023-07-12 12:04:00.000000"
# mac_module=os.environ['MAC_WEAR_DICT']['3480']
start="2023-08-17 09:52:00.000000"
end="2023-08-17 10:33:00.000000"
mac_module="DD84B2E50E06"
#ast.literal_eval(os.environ['MAC_WEAR_DICT'])['3480']


raw_data=callApi.getRawDataForCartoWear(mac_module,pd.to_datetime(start),pd.to_datetime(end))
print(raw_data)
#Preprocessing
sampling_time=2
window='10S'
preprocessor=Preprocessor(raw_data,sampling_time=sampling_time)
rssi_df=preprocessor.set_cleaner(remove_duplicates_Cleaner())\
        .cleaning()\
                .get_df()

INFO - data Cleaning


                   timestamp     macModule  rssi
0    2023-08-17 08:32:59.241  C45BBE39F9AE   -76
1    2023-08-17 08:32:58.387  C45BBE38FEAA   -62
2    2023-08-17 08:32:56.621  C45BBE39F9AE   -77
3    2023-08-17 08:32:55.089  C45BBE37B346   -78
4    2023-08-17 08:32:53.915  C45BBE39F9AE   -77
...                      ...           ...   ...
2633 2023-08-17 07:52:07.261  A8032A311DAA   -51
2634 2023-08-17 07:52:05.772  A8032A311F96   -79
2635 2023-08-17 07:52:02.407  A8032A311F56   -75
2636 2023-08-17 07:52:01.890  A8032A311DAA   -53
2637 2023-08-17 07:52:00.424  A8032A311F96   -78

[2638 rows x 3 columns]


pour tester la performance de notre modèle, on test de mettre moins de récepteurs pour voir comment notre système se comporte.

In [57]:
map_gdf
print(map_gdf["geometry"][0])
print(type(map_gdf["geometry"][0]))

POLYGON ((2.199773490428924 48.71280859384019, 2.199789583683014 48.71287053721895, 2.1998579800128937 48.71286345797952, 2.1998432278633118 48.71280085091248, 2.199773490428924 48.71280859384019))
<class 'shapely.geometry.polygon.Polygon'>


In [58]:
from src.DataLoader import room_dict_generator,room_r_tree_generator
room_dict=room_dict_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_dict)
from src.DataLoader import build_receiver_dict
receiver_dict= build_receiver_dict(marker_gdf,room_dict,room_r_tree)
from src.visualization import MapCreation

m=MapCreation(receiver_dict,room_dict)



In [59]:
#MacModule à enlever

# deleted_mac_module=['C45BBE39870E','A8032A311DAA','A8032A312052','C4BBE384CD2','C45BBE37B346','A8032A30FB9E','A8032A311F6A','A8032A311F96','A8032A31204E','C45BBE384CD2','C45BBE38FEAA']
# def macModuleDeleter(marker_gdf,rssi_df,deleted_mac_module):
#     for mac_module in deleted_mac_module:
#         marker_gdf=marker_gdf[marker_gdf.macModule != mac_module]
#     for mac_module in deleted_mac_module:
#         rssi_df=rssi_df[rssi_df.macModule != mac_module]
#     return marker_gdf,rssi_df
# marker_gdf,rssi_df=macModuleDeleter(marker_gdf,rssi_df,deleted_mac_module)


In [60]:
#to optimize the getting of location of macModule, we create a dictionnary
MacModuleLocation = marker_gdf.set_index("macModule").to_dict()["geometry"]
#it display the approximation of point location in print but it is not the case in the dataframe

### Bind the received RSSI Data to the CartoModule Marker on Map

In [61]:
#create a list of timestamps between min_timestamp and max_timestamp with a step of sampling_time
min_timestamp=rssi_df.iloc[0]['timestamp']+pd.Timedelta(sampling_time, unit="ms")
max_timestamp=rssi_df.iloc[-1]['timestamp']
timestamp_list= pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()


In [62]:
MarkerDictList=getMarkerDictList(rssi_df, timestamp_list)
MarkerDictList


[]

colormap creation

In [63]:
raw_data_features=RawDataFeatures(MarkerDictList,timestamp_list,MacModuleLocation)
MarkerLayer=TimestampedGeoJson(
                    {"type": "FeatureCollection","features": raw_data_features},
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,
                    auto_play=True)

In [64]:

MarkerLayer.add_to(m)
folium.LayerControl().add_to(m)
if saving_output:
    m.save('output/marker_map.html')
m

# implementation of low precision localisation

### 1. Naive implementation 

we're looking for 3 points at each sample,
if no 3 receivers received , we take the last 3 received,
if more than 3 receivers received we keep the 3 highest RSSI signal received

In [65]:
def max_marker_dict(marker_dict,edge_number=3):
    #keep only the edge_number modules with the highest rssi values
    #if there is less than edge_number modules, we keep all of them

    if len(marker_dict) > edge_number:
        # to do : function only if number of received signal for the module in list is 1 
        marker_dict={key: marker_dict[key] for key in sorted(marker_dict, key=marker_dict.get, reverse=True)[:edge_number]}
    return(marker_dict)
def getPolygonList(MarkerDictList,MacModuleLocation,min_edge_number=3,max_edge_number=3):
    #works with polygon with moe than 3 points but not with 3 points because it is not a convex polygon, should use convex hull
    list_of_list_of_points=[]
    for i in range(len(MarkerDictList)):
        marker_dict=MarkerDictList[i]
        list_of_points = []
        marker_dict=max_marker_dict(marker_dict,max_edge_number)
        for mac_module,rssi in marker_dict.items():
            list_of_points.append([MacModuleLocation[mac_module].x,MacModuleLocation[mac_module].y])
        
        count_backward=1
        while len(list_of_points) < min_edge_number and i-count_backward >= 0:

            points_to_add=3-len(list_of_points)
            marker_dict=MarkerDictList[i-count_backward]
            marker_dict=max_marker_dict(marker_dict,points_to_add)

            
            for mac_module,rssi in marker_dict.items():
                location=[MacModuleLocation[mac_module].x,MacModuleLocation[mac_module].y]
                if location in list_of_points:
                    continue
                list_of_points.append(location)
            count_backward+=1
        list_of_list_of_points.append(list_of_points)
    #eject the marker not having the right number of points ( not triangle for example)
    for index,list1 in enumerate(list_of_list_of_points):
        if len(list1)<min_edge_number:
            print("error")
            list_of_list_of_points.pop(index)
    return(list_of_list_of_points)
list_of_list_of_points=getPolygonList(MarkerDictList,MacModuleLocation,min_edge_number=3,max_edge_number=3)


list_of_list_of_points est une liste définissant les points associés aux 3 récepteurs les plus forts pour chaque temps. on cherche maintenant à afficher les polygones associés dans une map temporelle

In [66]:
def PolygonalFeatures(list_of_list_of_points,timestamp_list):
    input_format = '%Y-%m-%d %H:%M:%S.%f'
    output_format = '%Y-%m-%dT%H:%M:%S'
    features = []
    timestamp_list=[]
    for index in range(len(list_of_list_of_points)):
        timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
        dt = datetime.strptime(str(timestamp), input_format)
        new_timestamp = dt.strftime(output_format)
        timestamp_list.append(new_timestamp)
    features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': list_of_list_of_points
        },
        'properties': {
            'times': timestamp_list,
            'style': {'color': 'red', 'weight': 1},
            
        }
    }]
    return features


In [68]:
polygonal_features=PolygonalFeatures(list_of_list_of_points,timestamp_list)

from src.DataLoader import room_dict_generator,room_r_tree_generator
room_dict=room_dict_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_dict)
from src.DataLoader import build_receiver_dict
receiver_dict= build_receiver_dict(marker_gdf,room_dict,room_r_tree)
from src.visualization import MapCreation

m=MapCreation(receiver_dict,room_dict)

TimestampedGeoJson({"type": "FeatureCollection","features": polygonal_features},
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,  
                    auto_play=True).add_to(m)
folium.LayerControl().add_to(m)
m


In [69]:
def polygonlist_to_polygoncenterlist(list_of_list_of_points):
    centerList=[]
    for list1 in list_of_list_of_points:
        n=0
        xlist=0
        ylist=0
        for point in list1:    
            xlist+=point[0]
            ylist+=point[1]
            n+=1
        centerList.append([xlist/n,ylist/n])
    return(centerList)
centerList=polygonlist_to_polygoncenterlist(list_of_list_of_points)
        

In [70]:
centerList
for _center in centerList:
    if len(_center) != 2:
        print("error")

In [72]:
def CenterOfPolygonsFeatures(centerList,list_of_list_of_points,sampling_time):
    #different timestamp list
    input_format = '%Y-%m-%d %H:%M:%S.%f'
    output_format = '%Y-%m-%dT%H:%M:%S'
    timestamp_list=[]
    for index in range(len(list_of_list_of_points)):
        timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
        dt = datetime.strptime(str(timestamp), input_format)
        new_timestamp = dt.strftime(output_format)
        timestamp_list.append(new_timestamp)
        
    center_of_polygons_features = [{
        'type': 'Feature',
        'geometry': {
            'type': 'MultiPoint',
            'coordinates': centerList
        },
        'properties': {
            'times': timestamp_list,
            "icon": 'circle',
            'style': {'color': 'red', 'weight': 1},
            
        }
    }]
    return(center_of_polygons_features)
center_of_polygons_features=CenterOfPolygonsFeatures(centerList,list_of_list_of_points,sampling_time)
from src.DataLoader import room_dict_generator,room_r_tree_generator
room_dict=room_dict_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_dict)
from src.DataLoader import build_receiver_dict
receiver_dict= build_receiver_dict(marker_gdf,room_dict,room_r_tree)
from src.visualization import MapCreation

m=MapCreation(receiver_dict,room_dict)
TimestampedGeoJson(center_of_polygons_features,
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,  
                    auto_play=True).add_to(m)
folium.LayerControl().add_to(m)
if saving_output:
    m.save('output/map_with_point_locations.html')
m


In [73]:

import numpy as np
number_of_points=len(centerList)
longitude_data=np.empty(number_of_points)
latitude_data=np.empty(number_of_points)
for index,point in enumerate(centerList):
    longitude_data[index]=point[0]
    latitude_data[index]=point[1]
latitude_data,longitude_data

(array([], dtype=float64), array([], dtype=float64))

In [74]:
def kalman_filter(data, Q, R):
    n = len(data)
    x_est = np.zeros(n)
    P = np.zeros(n)
    x_est[0] = data[0]
    P[0] = 1.0

    for i in range(1, n):
        x_pred = x_est[i-1]
        P_pred = P[i-1] + Q

        K = P_pred / (P_pred + R)
        x_est[i] = x_pred + K * (data[i] - x_pred)
        print("diff=",x_est[i]-x_pred)
        P[i] = (1 - K) * P_pred

    return x_est

Q = 30  # Process noise
R = 10  # Measurement noise


In [75]:
def movingaveragetechnique(data,n=3):
    '''
    filtering the data with moving average technique


    https://en.wikipedia.org/wiki/Moving_average
    '''
    
    return np.convolve(data, np.ones(n), 'full') / n

In [76]:
longitude_filtered = movingaveragetechnique(longitude_data,10)
latitude_filtered = movingaveragetechnique(latitude_data,10)
print(longitude_filtered)
centerList_filtered=np.vstack((longitude_filtered,latitude_filtered)).T.tolist()
#attention , problême ici, on a pas le même nombre de timestamp que de points
while len(timestamp_list)>len(centerList_filtered):
    timestamp_list.pop()
timestamp_list=[]
input_format = '%Y-%m-%d %H:%M:%S.%f'
output_format = '%Y-%m-%dT%H:%M:%S'
for index in range(len(list_of_list_of_points)):
    timestamp = min_timestamp+index*pd.Timedelta(sampling_time, unit="ms")
    dt = datetime.strptime(str(timestamp), input_format)
    new_timestamp = dt.strftime(output_format)
    timestamp_list.append(new_timestamp)
print(len(centerList_filtered),len(timestamp_list))
print(centerList_filtered)
print(timestamp_list)

ValueError: v cannot be empty

In [ ]:
features = [{
    'type': 'Feature',
    'geometry': {
        'type': 'MultiPoint',
        'coordinates': centerList_filtered
    },
    'properties': {
        'times': timestamp_list,
        "icon": 'circle',
        'style': {'color': 'red', 'weight': 1},
        
    }
}]    
m = MapCreation(map_gdf,marker_gdf=marker_gdf)
TimestampedGeoJson(features,
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,  
                    auto_play=True).add_to(m)
folium.LayerControl().add_to(m)
if saving_output:
    m.save('output/map_with_point_locations_filtered.html')
m